In [69]:
import sys
from pathlib import Path
from subprocess import call
import glob
import os

In [39]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
from glob import glob
import os
from PIL import Image
import numpy as np
import pandas as pd
import MeCab
from wordcloud import WordCloud
import re
import os
import urllib.request

In [40]:
def convert_pdf_to_txt(path): # 引数にはPDFファイルパスを指定
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    laparams.detect_vertical = True # Trueにすることで綺麗にテキストを抽出できる
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    maxpages = 0
    caching = True
    pagenos=set()
    fstr = ''
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,caching=caching, check_extractable=True):
        interpreter.process_page(page)

        str = retstr.getvalue()
        fstr += str

    fp.close()
    device.close()
    retstr.close()
    return fstr

In [48]:
mecab = MeCab.Tagger ("-Owakati")
text = convert_pdf_to_txt(path).replace('\n', '')
text = mecab.parse (text)
pattern = r"[0-9A-Za-zぁ-んァ-ヴ一-龠]"
stopwords = re.sub(pattern, '',text)
# print(stopwords)

def wakati_by_mecab(text):
    tagger = MeCab.Tagger('')
    tagger.parse('') 
    node = tagger.parseToNode(text)
    word_list = []
    while node:
        pos = node.feature.split(",")[0]
        if pos in ["名詞"]:   # 対象とする品詞
            word = node.surface
            word_list.append(word)
        node = node.next
    return " ".join(word_list)

def lower_text(text):
    return text.lower()

def normalize_number(text):
    # 連続した数字を0で置換
    replaced_text = re.sub(r'\d+', '', text)
    return replaced_text

def download_stopwords(path):
    url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
    if os.path.exists(path):
        print('File already exists.')
    else:
        print('Downloading...')
        # Download the file from `url` and save it locally under `file_name`:
        urllib.request.urlretrieve(url, path)

def create_stopwords(path):
    stop_words = []
    for w in open(path, "r"):
        w = w.replace('\n','')
        if len(w) > 0:
          stop_words.append(w)
    return stop_words    

file_path = "/Users/yusuke-n/code/work/read_pdf/stopword.txt"
# text = wakati_by_mecab(text)
# text = lower_text(text)
# text = normalize_number(text)
download_stopwords(file_path)
stop_words = create_stopwords(file_path)

File already exists.


In [80]:
# path = "documents/JPA-1999069893.pdf"
# path = "documents/JPA-2007215290.pdf"
path = 'documents/JPA-2019187099.pdf'
files = glob.glob("./documents/*")
documents = []
for file in files:
    documents.append(file)

In [85]:
comments = []
for i in range(len(documents)):
    path = documents[i]
    mecab = MeCab.Tagger ("-Owakati")
    text = convert_pdf_to_txt(path).replace('\n', '')
    text = mecab.parse (text)
    pattern = r"[0-9A-Za-zぁ-んァ-ヴ一-龠]"
    stopwords = re.sub(pattern, '',text)
#     print(text)
    comments.append(text)
    sample_dict = {"comment": comments} 
    samples = pd.DataFrame.from_dict(sample_dict)
    samples["target"] = samples["comment"].apply(wakati_by_mecab).apply(lower_text).apply(normalize_number)

    count_vectorizer = CountVectorizer(stop_words=stop_words,token_pattern=u'(?u)\\b\\w+\\b')
    feature_vectors = count_vectorizer.fit_transform(samples['target'])
    vocabulary = count_vectorizer.get_feature_names() 
    
    wordcloud = WordCloud(
                      font_path="/system/Library/Fonts/ヒラギノ角ゴシック W4.ttc",
                      colormap='copper_r',
                      stopwords = set(stop_words),
                      collocations = False,
                      width=800,height=600).generate(samples['target'][i])
    
    basename=os.path.basename(documents[i])
    
    wordcloud.to_file("./{}.png".format(basename))

In [72]:
basename=os.path.basename('documents/JPA-2019187099.pdf')

'JPA-2019187099.pdf'

In [49]:
from sklearn.feature_extraction.text import CountVectorizer
comments = ["あなたは春が好きね", text]
sample_dict = {"comment": comments} 
samples = pd.DataFrame.from_dict(sample_dict)
samples["target"] = samples["comment"].apply(wakati_by_mecab).apply(lower_text).apply(normalize_number)

count_vectorizer = CountVectorizer(stop_words=stop_words,token_pattern=u'(?u)\\b\\w+\\b')
feature_vectors = count_vectorizer.fit_transform(samples['target'])
vocabulary = count_vectorizer.get_feature_names() 

In [53]:
from wordcloud import WordCloud
wordcloud = WordCloud(
                      font_path="/system/Library/Fonts/ヒラギノ角ゴシック W4.ttc",
                      colormap='copper_r',
                      stopwords = set(stop_words),
                      collocations = False,
                      width=800,height=600).generate(samples['target'][1])

In [54]:
wordcloud.to_file("./wordcloud_sample.png")

In [86]:
samples['target'][1]

'j p     -       a     .  .   (  )【 要約 課題 蓄電 装置 活用 こと 問題 なり 電力 系統 周波数 変化 抑制 需要 家 負荷 変動 当初 運用 計画 通り 分散 型 電源 運転 需給 制御 装置 方法 提供 こと 解決 手段 商用 系統 分散 型 電源  需要 家 需要 運転 制御 需給 制御 装置  つ 分散 型 電源 つ 需要 家 構成 グループ ， ｍ 電力 供給 電力 系統   前記 グループ つ 蓄電 装置  前記 蓄電 装置 制御 蓄電 制御 装置  前記 蓄電 制御 装置 前記 グループ 電力 潮流 指令 値 ｐｇ 前記 分散 型 電源 出力 指令 値 ｐｄ こと 特徴 需給 制御 装置 方法 選択 図 図  j p     -       a     .  .   (  )【 要約 課題 蓄電 装置 活用 こと 問題 なり 電力 系統 周波数 変化 抑制 需要 家 負荷 変動 当初 運用 計画 通り 分散 型 電源 運転 需給 制御 装置 方法 提供 こと 解決 手段 商用 系統 分散 型 電源  需要 家 需要 運転 制御 需給 制御 装置  つ 分散 型 電源 つ 需要 家 構成 グループ ， ｍ 電力 供給 電力 系統   前記 グループ つ 蓄電 装置  前記 蓄電 装置 制御 蓄電 制御 装置  前記 蓄電 制御 装置 前記 グループ 電力 潮流 指令 値 ｐｇ 前記 分散 型 電源 出力 指令 値 ｐｄ こと 特徴 需給 制御 装置 方法 選択 図 図  (  ) j p     -       a     .  .   商 用 系 統 ら 切 離 型 電 源 需 家 ら 需 要 応 運 転 制 需 給  型 電 源 有  需 要 家 構 成 グ ル ー 記 グ ル ー プ 設 ら  蓄 電 装 置 び 記 蓄 電 装 置 制 記 蓄 電 制 装 置 記 グ ル ー プ 電 力 潮 流 指 令 値 記 分 型 電 源 力 指 令 特 許 請 求 範 囲 請 求 項  制 装 置 プ 電 力 供 給 電 力 系 統 蓄 電 制 装 置 そ 値 請 求 項  こ 特 徴 需 給 制 装 置 請 求 項  記 需 給 制 装 置 記 グ ル ー プ 潮 流 指 令 値 前 記 蓄 電 制 装 置 記 電